# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np



# Import API key
from api_keys import geoapify_key


In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,63.32,94,94,21.45,SH,1735004294
1,1,waitangi,-43.9535,-176.5597,61.36,92,60,9.22,NZ,1735004295
2,2,ust'-charyshskaya pristan',52.3934,83.6635,-10.01,100,92,2.55,RU,1735004295
3,3,karamay,45.6004,84.8661,11.57,93,100,2.57,CN,1735004296
4,4,guerrero negro,27.9769,-114.0611,60.13,90,55,10.31,MX,1735004296


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
coordinates_df = city_data_df[['Lat','Lng','Humidity']]

map_plot_1 = coordinates_df.hvplot.points(
    'Lng',
    'Lat', 
    geo=True, 
    tiles="OSM",
    size = "Humidity")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 90
min_temp = 60

max_clouds = 75

brandons_ideal_spots = city_data_df[(city_data_df['Max Temp'] < max_temp) & (city_data_df['Max Temp'] > min_temp)
                                    & (city_data_df['Cloudiness'] < max_clouds)]

# Drop any rows with null values
brandons_ideal_spots = brandons_ideal_spots.dropna()

# Display sample data
brandons_ideal_spots

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,waitangi,-43.9535,-176.5597,61.36,92,60,9.22,NZ,1735004295
4,4,guerrero negro,27.9769,-114.0611,60.13,90,55,10.31,MX,1735004296
17,17,puerto baquerizo moreno,-0.9000,-89.6000,73.09,82,72,10.07,EC,1735004306
18,18,kerikeri,-35.2268,173.9474,74.88,53,3,9.55,NZ,1735004307
20,20,kudahuvadhoo,2.6708,72.8944,82.38,70,0,6.93,MV,1735004308
...,...,...,...,...,...,...,...,...,...,...
565,565,mount gambier,-37.8333,140.7667,71.29,47,64,7.96,AU,1735004687
566,566,coahuayana de hidalgo,18.7000,-103.6583,73.99,89,12,1.25,MX,1735004688
572,572,pisco,-13.7000,-76.2167,69.85,78,0,14.97,PE,1735004692
577,577,anna regina,7.2644,-58.5077,77.38,89,74,8.81,GY,1735004695


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = brandons_ideal_spots.loc[:,['City','Country',  'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,waitangi,NZ,-43.9535,-176.5597,92,NaN
4,guerrero negro,MX,27.9769,-114.0611,90,NaN
17,puerto baquerizo moreno,EC,-0.9000,-89.6000,82,NaN
18,kerikeri,NZ,-35.2268,173.9474,53,NaN
20,kudahuvadhoo,MV,2.6708,72.8944,70,NaN
...,...,...,...,...,...,...
565,mount gambier,AU,-37.8333,140.7667,47,NaN
566,coahuayana de hidalgo,MX,18.7000,-103.6583,89,NaN
572,pisco,PE,-13.7000,-76.2167,78,NaN
577,anna regina,GY,7.2644,-58.5077,89,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius =50000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address =  name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\distu\AppData\Local\Temp\ipykernel_39780\2551002128.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Hotel Chathams' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]


waitangi - nearest hotel: Hotel Chathams
guerrero negro - nearest hotel: Plaza sal paraiso
puerto baquerizo moreno - nearest hotel: Dory's House
kerikeri - nearest hotel: Avalon Resort
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
cooper's town - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
umluj - nearest hotel: No hotel found
kalmunai - nearest hotel: Saji new food
ribeira grande - nearest hotel: Hospedaria JSF
tolanaro - nearest hotel: Hôtel Mahavokey
hobyo - nearest hotel: Iftin Hotel Hobyo
san jose de feliciano - nearest hotel: Alojamiento Parador Erli
kertih - nearest hotel: Hotel Singgah
carnarvon - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
labuan bajo - nearest hotel: Zasgo Hotel
saipan - nearest hotel: Chalan Kanoa Beach Hotel
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
farafangana - nearest hotel: Le Concombre Masqué
zwedru - nearest hotel: No hotel found
quelimane - nearest hotel: Mab

,City,Country,Lat,Lng,Humidity,Hotel Name
1,waitangi,NZ,-43.9535,-176.5597,92,Hotel Chathams
4,guerrero negro,MX,27.9769,-114.0611,90,Plaza sal paraiso
17,puerto baquerizo moreno,EC,-0.9000,-89.6000,82,Dory's House
18,kerikeri,NZ,-35.2268,173.9474,53,Avalon Resort
20,kudahuvadhoo,MV,2.6708,72.8944,70,Hotel Riu Palace Maldivas
...,...,...,...,...,...,...
565,mount gambier,AU,-37.8333,140.7667,47,The Old Mount Gambier Gaol
566,coahuayana de hidalgo,MX,18.7000,-103.6583,89,Hotel Águila Real
572,pisco,PE,-13.7000,-76.2167,78,La Portada
577,anna regina,GY,7.2644,-58.5077,89,Jaigobin Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles='OSM', 
    hover_cols=['Hotel Name', 'Country']
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)